# MySwitzerland API

Swiss Destination data fetching
In this notebook, Destinations that stored in Myswitzerland is fetched with two main steps:
 - Create tables for stroing data
 - Get a list of destination field: name,identifier, latitude, longitude
There are two key aspects when fetching the data:
 - The request data is stored in the webpage, header is requred, thus `urllib.request.urlopen()`function is used to send url request
 - each request will return 10 entries, a for loop function is used to fetch through all pages that destination information is stored.
 

# Loading Packages

In [76]:
import requests
import re
import pandas as pd
import pprint
import json
import urllib.request, json

from time import gmtime, strftime
import time
import datetime
from time import sleep
import psycopg2
import config
import os.path

# Create table `Destinations`

In [79]:
# connect to postgres database
conn = psycopg2.connect(host="datalake-1.cyjrpnr3hv9x.us-east-1.rds.amazonaws.com",database="datalake1",user="MVision",password="MVision123")
cursor = conn.cursor()
conn.autocommit = True

def createDestinationsTable():
    cursor.execute("SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'destinations'")
    if cursor.fetchone()[0] != 1:
        #print('create table')
        cursor.execute(f"""
        CREATE TABLE "public"."destinations" (
          "dest_id" SERIAL PRIMARY KEY not null,
          "name" VARCHAR(255) null,
          "identifier" VARCHAR(255) UNIQUE not null,
          "latitude" double precision NULL,
          "longitude" double precision NULL);
          """)
        
createDestinationsTable()

In [15]:
# check page number
api_key = 'xVCUdgfAYA2xPlNDWWTWn6QCOTuvmUAz6xxHE6Vu'
url = 'https://opendata.myswitzerland.io/v1/destinations/'

hdr = { 'x-api-key' : api_key }

req = urllib.request.Request(url, headers=hdr)
response = urllib.request.urlopen(req)
data=json.loads(response.read())
pageCount=data['meta']['page']['totalPages']

for i in range(0,pageCount):
    curentUrl=url+'?page='+str(i)
    filename='locations-'+str(i)+'.json'
    #print(curentUrl)

https://opendata.myswitzerland.io/v1/destinations/?page=0
https://opendata.myswitzerland.io/v1/destinations/?page=1
https://opendata.myswitzerland.io/v1/destinations/?page=2
https://opendata.myswitzerland.io/v1/destinations/?page=3
https://opendata.myswitzerland.io/v1/destinations/?page=4
https://opendata.myswitzerland.io/v1/destinations/?page=5
https://opendata.myswitzerland.io/v1/destinations/?page=6
https://opendata.myswitzerland.io/v1/destinations/?page=7
https://opendata.myswitzerland.io/v1/destinations/?page=8
https://opendata.myswitzerland.io/v1/destinations/?page=9
https://opendata.myswitzerland.io/v1/destinations/?page=10
https://opendata.myswitzerland.io/v1/destinations/?page=11
https://opendata.myswitzerland.io/v1/destinations/?page=12
https://opendata.myswitzerland.io/v1/destinations/?page=13
https://opendata.myswitzerland.io/v1/destinations/?page=14
https://opendata.myswitzerland.io/v1/destinations/?page=15
https://opendata.myswitzerland.io/v1/destinations/?page=16
https:/

# Check Loading data

In [16]:
# get the data throuth endpoints
for field in data['data']:
    row=[
        field['name'],
        field['identifier'],
        field['geo']['latitude'],
        field['geo']['longitude']
    ]
    print(row)

['Lake Staz', 'fff733d5-7f3a-445f-908d-bc6436963855', 46.49853723585918, 9.83939900062391]
['Salez', 'ffd3f597-a217-4052-9d05-374fbfceb94d', 47.2347263556, 9.4996318422]
['Diegten', 'ff8df2f6-4419-4a25-b6ae-e313d1a6495e', 47.4135549828, 7.8118002222]
['Schwyz', 'ff8ccdf3-386b-48f2-8aa1-e73fa039685a', 47.02100936, 8.653659263]
['Stehrenberg', 'ff8bce7d-2622-44af-a67c-748602be56b5', 47.5230550758274, 9.08869743347168]
['Diessbach b.Büren', 'ff8b1fce-82c5-45d1-9948-5b80762874fe', 47.1088197372, 7.3615914058]
['Mont-Tramelan', 'ff74f36c-5e4c-4194-93bf-23441ca2e471', 47.20756532, 7.052893456]
['St. German', 'ff73cfe5-42f4-4830-ab07-b92d4bdb7322', 46.31243, 7.82185]
['La Joux FR', 'ff689406-a1f8-4236-8252-4d253282b17d', 46.6290843842, 6.9391720142]
['Arosa', 'ff5fdcde-4c42-4791-839e-9d566fe89eb6', 46.7784600772, 9.6771143822]


# Fetching Data and load to database

In [83]:
# connect to postgres database
conn = psycopg2.connect(host="datalake-1.cyjrpnr3hv9x.us-east-1.rds.amazonaws.com",database="datalake1",user="MVision",password="MVision123")
cursor = conn.cursor()
conn.autocommit = True

#api_key = 'xVCUdgfAYA2xPlNDWWTWn6QCOTuvmUAz6xxHE6Vu'
api_key = 'GBcoP22iWs72h6zpND0BdX5jGxy6hSkaU3oCtIxg'


base = 'https://opendata.myswitzerland.io/v1/destinations/'
hdr = { 'x-api-key' : api_key }
req = urllib.request.Request(base, headers=hdr)
response = urllib.request.urlopen(req)
data=json.loads(response.read())
pageCount=data['meta']['page']['totalPages']

fields=['name','identifier']
geos=['latitude','longitude']


for i in range(0,pageCount):
    curentUrl=base+'?page='+str(i)
    filename='locations-'+str(i)+'.json'
    print(curentUrl)
    req = urllib.request.Request(curentUrl, headers=hdr)
    
    with urllib.request.urlopen(req) as url:
        time.sleep(2)
        data = json.loads(url.read().decode())
    with open(filename, 'w') as f:
        json.dump(data, f)
    with open(filename, 'r') as json_file:
        data=json.load(json_file)
    #print(json.dumps(data['data'][0],indent=4))
        for location in data['data']:
            columns=[]
            row=[]
            for field in fields:
                if field in location:
                    columns.append(field)
                    row.append(location[field])
                
            if "geo" in location:
                print(location)
                for geo in geos:
                    if geo in location['geo']:
                        columns.append(geo)
                        row.append(location['geo'][geo])
            
            sql_columns=','.join(columns)
            sql_variable=','.join(['%s'] * len(columns))
            sql = f"INSERT INTO destinations ({sql_columns}) VALUES({sql_variable}) ON CONFLICT (identifier)DO NOTHING;"
            print(sql_columns)
            print(sql_variable)
            print(sql)
            cursor.execute(sql, tuple(row))     

https://opendata.myswitzerland.io/v1/destinations/?page=201
{'@context': 'https://schema.org/', '@type': 'TouristDestination', 'subjectOf': {'@type': 'CreativeWork', 'license': 'http://creativecommons.org/licenses/by-sa/4.0/'}, 'links': {'self': 'https://opendata.myswitzerland.io/v1/destinations//84712bcb-7917-4476-9ae7-5bc250fce090'}, 'identifier': '84712bcb-7917-4476-9ae7-5bc250fce090', 'name': 'Obstalden (Kerenzerberg)', 'url': 'https://www.myswitzerland.com/en/destinations/obstalden-kerenzerberg/', 'geo': {'@type': 'GeoCoordinates', 'latitude': 47.1169707231, 'longitude': 9.1510425647}, 'classification': [{'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'distancetoairport', 'values': [{'name': 'max2h', 'title': 'Max. 2 h'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'geographicallocations', 'values': [{'name': 'atthelake', 'title': 'At the lake'}, {'name': 'inthemountains', 'title': 'In the mountains'}, {'name': 'along

name,identifier
%s,%s
INSERT INTO destinations (name,identifier) VALUES(%s,%s) ON CONFLICT (identifier)DO NOTHING;
https://opendata.myswitzerland.io/v1/destinations/?page=202
{'@context': 'https://schema.org/', '@type': 'TouristDestination', 'subjectOf': {'@type': 'CreativeWork', 'license': 'http://creativecommons.org/licenses/by-sa/4.0/'}, 'links': {'self': 'https://opendata.myswitzerland.io/v1/destinations//83fad1d5-0473-4b0c-bb18-40b1ebaab483'}, 'identifier': '83fad1d5-0473-4b0c-bb18-40b1ebaab483', 'name': 'Bondasca and Albigna Valley', 'abstract': 'Though largely neglected by tourists in the past, this secluded area has long been an insider tip for climbers and mountaineers.', 'url': 'https://www.myswitzerland.com/en/destinations/bondasca-and-albigna-valley/', 'photo': 'https://www.myswitzerland.com/-/media/st/gadmin/images/landscapes/summer/mountains/bondasca1_202176.jpg', 'geo': {'@type': 'GeoCoordinates', 'latitude': 46.33756612979103, 'longitude': 9.646595755550864}, 'classific

{'@context': 'https://schema.org/', '@type': 'TouristDestination', 'subjectOf': {'@type': 'CreativeWork', 'license': 'http://creativecommons.org/licenses/by-sa/4.0/'}, 'links': {'self': 'https://opendata.myswitzerland.io/v1/destinations//82e7db7b-badd-4cbb-86ee-f26a7ebc9c1b'}, 'identifier': '82e7db7b-badd-4cbb-86ee-f26a7ebc9c1b', 'name': 'Couvet', 'url': 'https://www.myswitzerland.com/en/destinations/couvet/', 'geo': {'@type': 'GeoCoordinates', 'latitude': 46.9265226519, 'longitude': 6.6326601794}, 'classification': [{'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'geographicallocations', 'values': [{'name': 'alonggrandtour', 'title': 'Along the Grand Tour'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'licences', 'values': [{'name': 'bysa', 'title': 'BY-SA'}]}]}
name,identifier,latitude,longitude
%s,%s,%s,%s
INSERT INTO destinations (name,identifier,latitude,longitude) VALUES(%s,%s,%s,%s) ON CONFLICT (identifier)DO NOTHIN

{'@context': 'https://schema.org/', '@type': 'TouristDestination', 'subjectOf': {'@type': 'CreativeWork', 'license': 'http://creativecommons.org/licenses/by-sa/4.0/'}, 'links': {'self': 'https://opendata.myswitzerland.io/v1/destinations//822ec764-de95-4c45-88fd-fdcdc6c2aaef'}, 'identifier': '822ec764-de95-4c45-88fd-fdcdc6c2aaef', 'name': 'Zollbrück', 'url': 'https://www.myswitzerland.com/en/destinations/zollbrueck/', 'geo': {'@type': 'GeoCoordinates', 'latitude': 46.9753545469, 'longitude': 7.7416530108}, 'classification': [{'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'distancetoairport', 'values': [{'name': 'max1h', 'title': 'Max. 1 h'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'geographicallocations', 'values': [{'name': 'inthecountryside', 'title': 'In the countryside'}, {'name': 'alonggrandtour', 'title': 'Along the Grand Tour'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'lice

{'@context': 'https://schema.org/', '@type': 'TouristDestination', 'subjectOf': {'@type': 'CreativeWork', 'license': 'http://creativecommons.org/licenses/by-sa/4.0/'}, 'links': {'self': 'https://opendata.myswitzerland.io/v1/destinations//81b81746-9d65-45eb-a0a8-879a1ab8b531'}, 'identifier': '81b81746-9d65-45eb-a0a8-879a1ab8b531', 'name': 'Dorénaz', 'url': 'https://www.myswitzerland.com/en/destinations/dorenaz/', 'geo': {'@type': 'GeoCoordinates', 'latitude': 46.1473155714, 'longitude': 7.0436431536}, 'classification': [{'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'distancetoairport', 'values': [{'name': 'max3h', 'title': 'Max. 3 h'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'geographicallocations', 'values': [{'name': 'inthecountryside', 'title': 'In the countryside'}, {'name': 'alonggrandtour', 'title': 'Along the Grand Tour'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'licences'

...

https://opendata.myswitzerland.io/v1/destinations/?page=407
{'@context': 'https://schema.org/', '@type': 'TouristDestination', 'subjectOf': {'@type': 'CreativeWork', 'license': 'http://creativecommons.org/licenses/by-sa/4.0/'}, 'links': {'self': 'https://opendata.myswitzerland.io/v1/destinations//02b44434-4cf3-4b56-b273-d163a59c0663'}, 'identifier': '02b44434-4cf3-4b56-b273-d163a59c0663', 'name': 'Thun', 'abstract': 'With its historic buildings, towering raised promenades steeped in history, the white castle on the Schlossberg and the fantastic views over the lake set against the backdrop of the imposing Bernese Alps – Thun’s charm enchants locals and tourists alike. ', 'url': 'https://www.myswitzerland.com/en/destinations/thun-1/', 'photo': 'https://www.myswitzerland.com/-/media/dam/resources/places/t/h/thun/images%20winter/58339_32001800.jpeg', 'geo': {'@type': 'GeoCoordinates', 'latitude': 46.7579824772, 'longitude': 7.6284545597}, 'classification': [{'@context': 'https://myswitzerl

{'@context': 'https://schema.org/', '@type': 'TouristDestination', 'subjectOf': {'@type': 'CreativeWork', 'license': 'http://creativecommons.org/licenses/by-sa/4.0/'}, 'links': {'self': 'https://opendata.myswitzerland.io/v1/destinations//025c39a4-046d-4f7f-a8ed-fd9d9d1d80c0'}, 'identifier': '025c39a4-046d-4f7f-a8ed-fd9d9d1d80c0', 'name': 'Carabbia', 'url': 'https://www.myswitzerland.com/en/destinations/carabbia/', 'geo': {'@type': 'GeoCoordinates', 'latitude': 45.9724635461, 'longitude': 8.9363788683}, 'classification': [{'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'geographicallocations', 'values': [{'name': 'alonggrandtour', 'title': 'Along the Grand Tour'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'licences', 'values': [{'name': 'bysa', 'title': 'BY-SA'}]}]}
name,identifier,latitude,longitude
%s,%s,%s,%s
INSERT INTO destinations (name,identifier,latitude,longitude) VALUES(%s,%s,%s,%s) ON CONFLICT (identifier)DO NO

{'@context': 'https://schema.org/', '@type': 'TouristDestination', 'subjectOf': {'@type': 'CreativeWork', 'license': 'http://creativecommons.org/licenses/by-sa/4.0/'}, 'links': {'self': 'https://opendata.myswitzerland.io/v1/destinations//0216cd51-dee2-43e6-9836-8cc54ba97538'}, 'identifier': '0216cd51-dee2-43e6-9836-8cc54ba97538', 'name': 'Molare', 'url': 'https://www.myswitzerland.com/en/destinations/molare/', 'geo': {'@type': 'GeoCoordinates', 'latitude': 46.4864560425524, 'longitude': 8.82914543151856}, 'classification': [{'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'geographicallocations', 'values': [{'name': 'alonggrandtour', 'title': 'Along the Grand Tour'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'licences', 'values': [{'name': 'bysa', 'title': 'BY-SA'}]}]}
name,identifier,latitude,longitude
%s,%s,%s,%s
INSERT INTO destinations (name,identifier,latitude,longitude) VALUES(%s,%s,%s,%s) ON CONFLICT (identifier)DO

{'@context': 'https://schema.org/', '@type': 'TouristDestination', 'subjectOf': {'@type': 'CreativeWork', 'license': 'http://creativecommons.org/licenses/by-sa/4.0/'}, 'links': {'self': 'https://opendata.myswitzerland.io/v1/destinations//01491d23-296b-4a2f-af1d-cc108ef6e455'}, 'identifier': '01491d23-296b-4a2f-af1d-cc108ef6e455', 'name': 'Orselina', 'url': 'https://www.myswitzerland.com/en/destinations/orselina/', 'geo': {'@type': 'GeoCoordinates', 'latitude': 46.1802055425, 'longitude': 8.8013987153}, 'classification': [{'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'geographicallocations', 'values': [{'name': 'alonggrandtour', 'title': 'Along the Grand Tour'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'licences', 'values': [{'name': 'bysa', 'title': 'BY-SA'}]}]}
name,identifier,latitude,longitude
%s,%s,%s,%s
INSERT INTO destinations (name,identifier,latitude,longitude) VALUES(%s,%s,%s,%s) ON CONFLICT (identifier)DO NO

{'@context': 'https://schema.org/', '@type': 'TouristDestination', 'subjectOf': {'@type': 'CreativeWork', 'license': 'http://creativecommons.org/licenses/by-sa/4.0/'}, 'links': {'self': 'https://opendata.myswitzerland.io/v1/destinations//009acbeb-34f5-44fa-8235-5562161f46d3'}, 'identifier': '009acbeb-34f5-44fa-8235-5562161f46d3', 'name': 'Lostorf', 'url': 'https://www.myswitzerland.com/en/destinations/lostorf/', 'geo': {'@type': 'GeoCoordinates', 'latitude': 47.3819280156, 'longitude': 7.9469960025}, 'classification': [{'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'distancetoairport', 'values': [{'name': 'max1h', 'title': 'Max. 1 h'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'geographicallocations', 'values': [{'name': 'inthecountryside', 'title': 'In the countryside'}]}, {'@context': 'https://myswitzerland.io/', '@type': 'Classification', 'name': 'licences', 'values': [{'name': 'bysa', 'title': 'BY-SA'}]}, {'@context

In [ ]:
for location in data['data']:
            columns=[]
            row=[]
            for field in fields:
                if field in location:
                    columns.append(field)
                    row.append(location[field])
                
            if "geo" in location:
                print(location)
                for geo in geos:
                    if geo in location['geo']:
                        columns.append(geo)
                        row.append(location['geo'][geo])
            
            sql_test=''
            for column in columns:
                sql_test+=', '+column
            print(sql_test)
            
            sql_columns=','.join(columns)
            sql_variable=','.join(['%s'] * len(columns))
            sql = f"INSERT INTO destinations ({sql_columns}) VALUES({sql_variable})"
            print(sql_columns)
            print(sql_variable)
            print(sql)
            #cursor.execute(sql, tuple(row))     